In [1]:
from common import g2json as gd
import Quandl as q
import pandas as pd
from datetime import date 
from datetime import timedelta
import logging
import sys
import MySQLdb

FORMAT = '%(asctime)-15s %(levelname)s %(message)s'
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [2]:
def get_symbols():
    """
    -----------------------------------------------------------------------
        look up symbols from Google Doc
    -----------------------------------------------------------------------
    """
    key = "1Vbgw4yw1crruuH05aDSHXBvD_Bb_Tq47Fs04MNJVDk4"
    sheet = "Symbols"

    symbols = pd.read_json(gd.get_gdoc(key, sheet, return_json=True))

    return symbols

In [3]:
def get_data(symbols, start_date):

    """ 
    -----------------------------------------------------------------------
        gets Quandl data for each symbol
    ----------------------------------------------------------------------- 
    """ 

    data = None
    for i, row in symbols.iterrows():

        month = "1"
        search = "{0}/{1}_{2}{3}".format(row.database, row.source, row.symbol, month)

        frequency = "daily"

        logger.info('requesting {0} data for {1} from {2} on from quandl'.format(frequency, row.symbol, start_date))	

        df = q.get(search, authtoken="xkXtzxzRcDyizWAV9r3_", 
                   collapse=frequency,
                   sort_order="asc", 
                   trim_start=start_date)

        df['Symbol'] = row.symbol
        df['Code'] = row.code
        if 'Open Interest' in df.columns:
            df=df.rename(columns = {'Open Interest':'OI'})
        
        if 'Prev. Day Open Interest' in df.columns:
            df=df.rename(columns = {'Prev. Day Open Interest':'Prev_OI'})


        if (data is not None):        
            data = data.append(df, ignore_index=False)
            logger.info('appending {0} {1}, {2} rows ({3})'.format(row.contractname, df.Symbol[0], len(df.index), len(data.index)))
        else:
            # first time around?
            data = df
            logger.info('initializing {0} {1}, {2} rows ({3})'.format(row.contractname, df.Symbol[0], len(df.index), len(data.index)))     

    return data   

In [4]:
def get_db():

    # change your server, login and db
    return MySQLdb.connect("localhost","root","","calogica" )

In [5]:
def trunc_data(data, table_name, key, index):

    groups = data.groupby(key)

    db = get_db()
    c = db.cursor()
    for name, group in groups:

        index_start = group.index.min().date()
        del_cmd = "delete from {0} where {1} ='{2}' and {3} >= '{4}'".format(table_name, key, name, index, index_start)
        print del_cmd

        c.execute(del_cmd)
        db.commit()
        
    c.close()
    db.close()

In [6]:
def save_data(data, table_name, index, append=False):

    db = get_db()
    if db:
        if append:
            data.to_sql(con=db, name=table_name, index=index, if_exists='append', flavor='mysql')

        else:

            data.to_sql(con=db, name=table_name, index=index, if_exists='replace', flavor='mysql')

        # magically all the dataz are in there
        tbl = pd.read_sql('select * from {0};'.format(table_name), con=db)

        db.close()

    return tbl

In [7]:
trading_days = 100.0
days = trading_days/5.0*7
d = date.today() - timedelta(days=days)
start_date = d.strftime('%Y-%m-%d')
start_date

'2014-12-13'

In [8]:
symbols = get_symbols()

In [9]:
db = get_db()
db.close()

In [10]:
save_data(symbols, "symbols", False, False)

/Users/claus/venv/lib/python2.7/site-packages/pandas/io/sql.py:588: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy engines.
  warnings.warn(_MYSQL_WARNING, FutureWarning)


,__rowid__,code,contractmonths,contractname,database,exchange,market,source,symbol
0,cokwr,ICE_CC,HKNUZ,Cocoa Futures,CHRIS,ICE,Agricultural,ICE,CC
1,cpzh4,ICE_KC,HKNUZ,Coffee C Futures,CHRIS,ICE,Agricultural,ICE,KC
2,cre1l,CME_C,HKNUZ,Corn Futures,CHRIS,CBT,Agricultural,CME,C
3,chk2m,ICE_CT,HKNVZ,Cotton No. 2 Futures,CHRIS,ICE,Agricultural,ICE,CT
4,ciyn3,CME_FC,FHJKQUVX,Feeder Cattle Futures,CHRIS,CME,Agricultural,CME,FC
5,ckd7g,CME_LN,GJKMNQVZ,Lean Hog Futures,CHRIS,CME,Agricultural,CME,LN
6,clrrx,CME_LC,GJMQVZ,Live Cattle Futures,CHRIS,CME,Agricultural,CME,LC
7,cyevm,CME_O,HKNUZ,Oats Futures,CHRIS,CBT,Agricultural,CME,O
8,cztg3,ICE_OJ,FHKNUX,Orange Juice Futures,CHRIS,ICE,Agricultural,ICE,OJ
9,d180g,CME_LB,FHKNUX,Random Length Lumber Futures,CHRIS,CME,Agricultural,CME,LB


In [11]:
data = get_data(symbols, start_date)

2015-05-02 05:40:23,869 - __main__ - INFO - requesting daily data for CC from 2014-12-13 on from quandl
INFO:__main__:requesting daily data for CC from 2014-12-13 on from quandl
2015-05-02 05:40:24,211 - __main__ - INFO - initializing Cocoa Futures CC, 95 rows (95)
INFO:__main__:initializing Cocoa Futures CC, 95 rows (95)
2015-05-02 05:40:24,212 - __main__ - INFO - requesting daily data for KC from 2014-12-13 on from quandl
INFO:__main__:requesting daily data for KC from 2014-12-13 on from quandl
2015-05-02 05:40:24,510 - __main__ - INFO - appending Coffee C Futures KC, 95 rows (190)
INFO:__main__:appending Coffee C Futures KC, 95 rows (190)
2015-05-02 05:40:24,512 - __main__ - INFO - requesting daily data for C from 2014-12-13 on from quandl
INFO:__main__:requesting daily data for C from 2014-12-13 on from quandl
2015-05-02 05:40:24,804 - __main__ - INFO - appending Corn Futures C, 96 rows (286)
INFO:__main__:appending Corn Futures C, 96 rows (286)
2015-05-02 05:40:24,805 - __main__ -

In [12]:
table_name = "prices"
key="Code"
index="Date"

In [13]:
data

,Block Volume,Change,Code,EFP Volume,EFS Volume,High,Last,Low,Open,Prev_OI,Settle,Symbol,Volume,Wave
Date,,,,,,,,,,,,,,
2014-12-15,NaN,16.00000,ICE_CC,276,NaN,2884.00000,NaN,2852.00000,2862.00000,90670,2873.00000,CC,7256,2868
2014-12-16,NaN,41.00000,ICE_CC,158,NaN,2926.00000,NaN,2885.00000,2890.00000,90374,2914.00000,CC,10748,2905
2014-12-17,NaN,1.00000,ICE_CC,113,NaN,2931.00000,NaN,2890.00000,2919.00000,91166,2915.00000,CC,8223,2913
2014-12-18,NaN,50.00000,ICE_CC,1434,NaN,2968.00000,NaN,2919.00000,2922.00000,91329,2965.00000,CC,16370,2953
2014-12-19,NaN,15.00000,ICE_CC,76,NaN,2990.00000,NaN,2952.00000,2965.00000,91163,2980.00000,CC,13020,2976
2014-12-22,NaN,6.00000,ICE_CC,311,NaN,2996.00000,NaN,2970.00000,2995.00000,92092,2986.00000,CC,6506,2986
2014-12-23,NaN,-20.00000,ICE_CC,489,NaN,2985.00000,NaN,2954.00000,2974.00000,92982,2966.00000,CC,6632,2970
2014-12-24,NaN,-2.00000,ICE_CC,540,NaN,2997.00000,NaN,2956.00000,2976.00000,93330,2964.00000,CC,3462,2981
2014-12-26,NaN,NaN,ICE_CC,NaN,NaN,2964.00000,NaN,2935.00000,2964.00000,93829,2941.00000,CC,2906,2945


In [14]:
table_name = "prices"
key="Code"
index="Date"
for i, row in symbols.iterrows():
    df_sym = data[data["Code"] == row['code']]
    trunc_data(df_sym, table_name, key, index)
    df_db = save_data(df_sym, table_name, index=True, append=True)

delete from prices where Code ='ICE_CC' and Date >= '2014-12-15'
delete from prices where Code ='ICE_KC' and Date >= '2014-12-15'
delete from prices where Code ='CME_C' and Date >= '2014-12-15'
delete from prices where Code ='ICE_CT' and Date >= '2014-12-15'
delete from prices where Code ='CME_FC' and Date >= '2014-12-15'
delete from prices where Code ='CME_LN' and Date >= '2014-12-15'
delete from prices where Code ='CME_LC' and Date >= '2014-12-15'
delete from prices where Code ='CME_O' and Date >= '2014-12-15'
delete from prices where Code ='ICE_OJ' and Date >= '2014-12-15'
delete from prices where Code ='CME_LB' and Date >= '2014-12-15'
delete from prices where Code ='CME_S' and Date >= '2014-12-15'
delete from prices where Code ='CME_SM' and Date >= '2014-12-15'
delete from prices where Code ='CME_BO' and Date >= '2014-12-15'
delete from prices where Code ='ICE_SB' and Date >= '2014-12-15'
delete from prices where Code ='CME_W' and Date >= '2014-12-15'
delete from prices where Code

/Users/claus/venv/lib/python2.7/site-packages/pandas/io/sql.py:1309: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  warnings.warn(_SAFE_NAMES_WARNING)
